# Provision data quality report
**Author**:  Greg Slater <br>
**Date created**:  April 2025 <br>
**Dataset Scope**: ODP datasets <br>
**Report Type**: Ad-hoc <br>

**Purpose**: The purpose of this report is demonstrate the basic methodology of scoring provisions using the data quality measurement framework. The data sources required for assessing whether a provision passes or fails criteria in the framework vary, so currently this stripped back method just uses issues logged in the `digital-land.issue` table which are summarised in `performance.endpoint_dataset_issue_type_summary`. The starting point is the `performance.provision_summary` table, which includes all provisions and whether they currently have an active endpoint or not. For simplicity only provisions with one or more active endpoints are scored.

### Data quality framework  
The table below visualises the framework that is used to assign a quality level to each ODP data provision. 

The criteria marked as "true" at each level must be met by a data provision in order for it to be scored at that level. The levels are cumulative, so all criteria must be met in order for a provision to be scored as *data that is trustworthy*.

![quality framework table](quality-framework-table.png)



Criteria not included in this example method are:

* *Data from an authoritative source*
* *No deleted entities*
* *Conservation area entity count matches LPA*
* *Entities within LPA boundary*

In [14]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

import functions_core as fc

td = datetime.today().strftime('%Y-%m-%d')

db_dir = "../../data/db_downloads/"
os.makedirs(db_dir, exist_ok=True)

output_dir = "../../data/quality_report/"
os.makedirs(output_dir, exist_ok=True)

## 1. Import

In [15]:
# performance db
fc.download_dataset("performance", db_dir, overwrite=False)
path_perf_db = os.path.join(db_dir, "performance.db")

# Issue quality criteria lookup
lookup_issue_qual = fc.datasette_query(
    "digital-land",
    """
    SELECT 
        description,
        issue_type,
        name,
        severity,
        responsibility,
        quality_criteria_level || " - " || quality_criteria as quality_criteria,
        quality_criteria_level as quality_level
    FROM issue_type
    WHERE quality_criteria_level IS NOT NULL
    """ 
    )


## 2. Transform

In [16]:
provision = fc.query_sqlite(
    path_perf_db,
    """
    SELECT organisation, dataset, active_endpoint_count
    FROM provision_summary
""")

# provision

In [17]:
# extract issue count by provision from endpoint_dataset_issue_type_summary
qual_issue = fc.query_sqlite(
    path_perf_db,
    """
    SELECT 
        organisation, dataset,
        'issue' as problem_source,
        issue_type as problem_type, 
        sum(count_issues) as count
    FROM endpoint_dataset_issue_type_summary
    WHERE resource_end_date is not NULL
    AND issue_type is not NULL
    GROUP BY organisation, dataset, issue_type
""")

# join on quality criteria and level from issue_type lookup (this restricts to only issues linked to a quality criteria)
qual_issue = qual_issue.merge(
    lookup_issue_qual[["issue_type", "quality_criteria", "quality_level"]],
    how = "inner",
    left_on = "problem_type",
    right_on = "issue_type"
)

qual_issue.drop("issue_type", axis=1, inplace=True)

qual_issue.to_csv("01_quality_problems-source-issues.csv", index = False)
# qual_issue

In [18]:
# combine all problem source tables, and aggregate to criteria level

qual_all_criteria = pd.concat(
    [qual_issue]
).groupby(
    ["organisation", "dataset", "quality_criteria", "quality_level"],
    as_index=False
).agg(
    count_failures = ("count", "sum")
)

# qual_all_criteria

In [19]:
prov_qual_all = provision.merge(
    qual_all_criteria,
    how = "left",
    on = ["organisation", "dataset"]
)

prov_qual_all.to_csv("02_provision_quality_all_criteria.csv", index = False)
# prov_qual_all

In [20]:
# when quality_level is null & active_endpoint_count > 0 then quality_level = 4 (we have endpoints with no issues, so quality = trustworthy)
# active_endpoint_count = 0 then quality_level = 0 (we have no active endpoints, so quality = no score)
prov_qual_all["quality_level_comp"] = np.select(
    [
        (prov_qual_all["active_endpoint_count"] == 0),
        (prov_qual_all["quality_level"].notnull()),
        (prov_qual_all["active_endpoint_count"] > 0) & (prov_qual_all["quality_level"].isnull())
    ],
    [
        0, prov_qual_all["quality_level"], 4
    ]
)

In [21]:
level_map = {
    4: "4. data that is trustworthy",
    3: "3. data that is good for ODP",
    2: "2. authoritative data from the LPA",
    1: "1. some data",
    0: "0. no score"}

prov_quality = prov_qual_all.groupby([
    "organisation", "dataset"
],
    as_index=False,
    dropna=False
).agg(
    quality_level = ("quality_level_comp", "min")
)

prov_quality["quality"] = prov_quality["quality_level"].map(level_map)
prov_quality["notes"] = ""
prov_quality["end-date"] = ""
prov_quality["start-date"] = td
prov_quality["entry-date"] = td

print(prov_quality.value_counts(["quality_level", "quality"]))
prov_quality[["dataset", "end-date", "entry-date", "notes", "organisation", "quality", "start-date"]].to_csv("02_provision_quality_scored.csv", index = False)

quality_level  quality                           
0.0            0. no score                           4956
4.0            4. data that is trustworthy            387
3.0            3. data that is good for ODP           237
2.0            2. authoritative data from the LPA     177
Name: count, dtype: int64
